In [7]:
import pandas as pd
from sklearn.metrics import root_mean_squared_error
import sys
sys.path.append('C:/GitHub/rag/app')  # Adjust the path to the parent directory containing the 'app' module
from app.rag_pipelines import NoRagLLM
from tqdm import tqdm

In [8]:
df_1 = pd.read_json('../PROPHET/data_2024-8/dataset_L1.json')
df_2 = pd.read_json('../PROPHET/data_2024-8/dataset_L2.json')

In [9]:
df_1

,total_predictors,community_prediction,rule_details,categories,background,author,Estimated_resolved_time,date_begin,date_close,articles,search_queries,id,source,question,resolution_criteria,date,answer,topic,human_answer,url
0,28,14%,The question resolves as the overall risk. The...,[Q3 AI Forecasting Benchmark Tournament],"In evolving situations, experts at the U.S. Ce...",Tom_Metaculus,2024-09-13,2024-08-23,2024-08-24,"[{'id': 83, 'url': 'https://www.thelancet.com/...","[CDC mpox risk assessment update August 2024, ...",0,Metaculus,Will the CDC's assessment of the risk posed by...,The question resolves Yes if the risk to the g...,2024-08-24,1,NaN,NaN,NaN
1,28,15%,The question resolves as the overall risk. The...,[Q3 AI Forecasting Benchmark Tournament],"In evolving situations, experts at the U.S. Ce...",Tom_Metaculus,2024-10-03,2024-08-23,2024-08-24,"[{'id': 69, 'url': 'https://tools.cdc.gov/podc...","[CDC mpox risk assessment updates August 2024,...",1,Metaculus,Will the CDC's assessment of the risk posed by...,The question resolves Yes if the risk to the g...,2024-08-24,0,NaN,NaN,NaN
2,27,38%,None,[Q3 AI Forecasting Benchmark Tournament],Where have OpenAI’s founders gone?Just two of ...,Tom_Metaculus,2024-10-03,2024-08-23,2024-08-24,"[{'id': 43, 'url': 'https://menafn.com/arabic/...","[Greg Brockman OpenAI return news August 2024,...",4,Metaculus,Will Greg Brockman return to OpenAI following ...,This question resolves as Yes if Greg Brockman...,2024-08-24,0,NaN,NaN,NaN
3,110,99%,An announced suspension of campaigning that is...,"[Elections, Politics, Robert F. Kennedy Jr., U...",The 2024 United States presidential election i...,Jgalt,2024-10-21,2024-04-12,2024-08-24,"[{'id': 99, 'url': 'https://www.cnn.com/2023/0...",[Robert F. Kennedy Jr. 2024 presidential campa...,9,Metaculus,Will Robert F. Kennedy Jr. suspend his 2024 pr...,This question resolves as Yes if on or before ...,2024-08-24,1,NaN,NaN,NaN
4,32,54%,The Warren Buffett indicator will be considere...,[Q3 AI Forecasting Benchmark Tournament],In a 2001 article co-written with Fortune Maga...,johnnycaffeine,2024-09-18,2024-08-20,2024-08-21,"[{'id': 41, 'url': 'https://economictimes.indi...","[Warren Buffett Indicator current status 2024,...",23,Metaculus,Will the Warren Buffett Indicator exceed 200% ...,This question resolves as Yes if before Septem...,2024-08-21,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,NaN,NaN,NaN,NaN,,NaN,NaT,NaN,NaN,"[{'id': 4, 'url': 'https://abcnews.go.com/Busi...","[Linda Yaccarino CEO X news April 2024, Linda ...",137,Manifold,Will Linda Yaccarino be the CEO of X on April ...,Linda Yaccarino is the CEO of X. If she no lon...,2024-05-31,1,Technology,0.97,https://manifold.markets/KevinRoose18ac/will-l...
95,33,65%,If Keir Starmer is not the Prime Minister of t...,[Q3 AI Forecasting Benchmark Tournament],Keir Starmer recently became the Prime Ministe...,Tom_Metaculus,2024-09-28,2024-08-16,2024-08-17,"[{'id': 15, 'url': 'https://www.independent.co...","[Keir Starmer approval rating September 2024, ...",139,Metaculus,Will Keir Starmer's approval rating as Prime M...,The question will resolve Yes if the numerical...,2024-08-17,1,NaN,NaN,NaN
96,32,83%,If Keir Starmer is not the Prime Minister of t...,[Q3 AI Forecasting Benchmark Tournament],Keir Starmer recently became the Prime Ministe...,Tom_Metaculus,2024-09-28,2024-08-16,2024-08-17,"[{'id': 17, 'url': 'https://www.irishtimes.com...",[Keir Starmer approval rating September 2024 n...,141,Metaculus,Will Keir Starmer's approval rating as Prime M...,The question will resolve Yes if the numerical...,2024-08-17,1,NaN,NaN,NaN
97,33,30%,The relevant Fed policy rate is the federal fu...,[Q3 AI Forecasting Benchmark Tournament],"In early August 2024, bond markets have been r...",Tom_Metaculus,2024-09-19,2024-08-22,2024-08-23,"[{'id': 36, 'url': 'https://news.yahoo.com/fed...","[Fed interest rate decision September 2024, Fe...",145,Metaculus,Will the

In [10]:
# Select the columns whwre community prediction is not none and not empty string
df_test = df_1[df_1['community_prediction'].notna()]
# Select the columns where community prediction is not empty string
df_test = df_test[df_test['community_prediction'] != '']

#Make community prediction from str "...%" to int
df_test['community_prediction'] = df_test['community_prediction'].str.replace('%', '')
df_test['community_prediction'] = df_test['community_prediction'].astype(int)


df_test

,total_predictors,community_prediction,rule_details,categories,background,author,Estimated_resolved_time,date_begin,date_close,articles,search_queries,id,source,question,resolution_criteria,date,answer,topic,human_answer,url
0,28,14,The question resolves as the overall risk. The...,[Q3 AI Forecasting Benchmark Tournament],"In evolving situations, experts at the U.S. Ce...",Tom_Metaculus,2024-09-13,2024-08-23,2024-08-24,"[{'id': 83, 'url': 'https://www.thelancet.com/...","[CDC mpox risk assessment update August 2024, ...",0,Metaculus,Will the CDC's assessment of the risk posed by...,The question resolves Yes if the risk to the g...,2024-08-24,1,NaN,NaN,NaN
1,28,15,The question resolves as the overall risk. The...,[Q3 AI Forecasting Benchmark Tournament],"In evolving situations, experts at the U.S. Ce...",Tom_Metaculus,2024-10-03,2024-08-23,2024-08-24,"[{'id': 69, 'url': 'https://tools.cdc.gov/podc...","[CDC mpox risk assessment updates August 2024,...",1,Metaculus,Will the CDC's assessment of the risk posed by...,The question resolves Yes if the risk to the g...,2024-08-24,0,NaN,NaN,NaN
2,27,38,None,[Q3 AI Forecasting Benchmark Tournament],Where have OpenAI’s founders gone?Just two of ...,Tom_Metaculus,2024-10-03,2024-08-23,2024-08-24,"[{'id': 43, 'url': 'https://menafn.com/arabic/...","[Greg Brockman OpenAI return news August 2024,...",4,Metaculus,Will Greg Brockman return to OpenAI following ...,This question resolves as Yes if Greg Brockman...,2024-08-24,0,NaN,NaN,NaN
3,110,99,An announced suspension of campaigning that is...,"[Elections, Politics, Robert F. Kennedy Jr., U...",The 2024 United States presidential election i...,Jgalt,2024-10-21,2024-04-12,2024-08-24,"[{'id': 99, 'url': 'https://www.cnn.com/2023/0...",[Robert F. Kennedy Jr. 2024 presidential campa...,9,Metaculus,Will Robert F. Kennedy Jr. suspend his 2024 pr...,This question resolves as Yes if on or before ...,2024-08-24,1,NaN,NaN,NaN
4,32,54,The Warren Buffett indicator will be considere...,[Q3 AI Forecasting Benchmark Tournament],In a 2001 article co-written with Fortune Maga...,johnnycaffeine,2024-09-18,2024-08-20,2024-08-21,"[{'id': 41, 'url': 'https://economictimes.indi...","[Warren Buffett Indicator current status 2024,...",23,Metaculus,Will the Warren Buffett Indicator exceed 200% ...,This question resolves as Yes if before Septem...,2024-08-21,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,32,60,Resolves based on reporting from Tesla's Inves...,[Q3 AI Forecasting Benchmark Tournament],"Since the beginning of 2023, Tesla's productio...",johnnycaffeine,2024-10-03,2024-08-02,2024-08-03,"[{'id': 47, 'url': 'https://www.perthnow.com.a...","[Tesla production forecast Q3 2024, Tesla prod...",132,Metaculus,Will Tesla increase its production in Q3 2024 ...,This question resolves as Yes if Tesla reports...,2024-08-03,1,NaN,NaN,NaN
92,14,1,"The type of medal does not matter, nor does it...",[Sports & Entertainment],Tennis at the Olympics has five events: single...,BrunoParga,2024-08-05,2024-07-28,2024-08-04,"[{'id': 57, 'url': 'https://www.brisbanetimes....","[2024 Paris Olympics tennis events schedule, t...",134,Metaculus,Will an athlete win more than one medal for te...,This question resolves as Yes if any athlete w...,2024-08-04,0,NaN,NaN,NaN
95,33,65,If Keir Starmer is not the Prime Minister of t...,[Q3 AI Forecasting Benchmark Tournament],Keir Starmer recently became the Prime Ministe...,Tom_Metaculus,2024-09-28,2024-08-16,2024-08-17,"[{'id': 15, 'url': 'https://www.independent.co...","[Keir Starmer approval rating September 2024, ...",139,Metaculus,Will Keir Starmer's approval rating as Prime M...,The question will resolve Yes if the numerical...,2024-08-17,1,NaN,NaN,NaN
96,32,83,If Keir Starmer is not the Prime Minister of t...,[Q3 AI Forecasting Benchmark Tournament],Keir Starmer recently became the Prime Ministe...,Tom_Metaculus,2024-09-28,2024-08-16,2024-08-17,"[{'id': 17, 'url': 'https://www.irishtim

In [11]:
def compute_metrics(df, rag):
    true_values = df['community_prediction'].tolist()
    predictions = []

    for index, row in tqdm(df.iterrows(), total=len(df)):
        # Get the community prediction and the actual value
        question = row['question']
        while True:
            try:
                # Get the prediction from the RAG model
                prediction = rag.generate_anwser(question)["probability"]
                break
            except Exception as e:
                print(f"Error: {e}. Retrying...")

        predictions.append(prediction)

    return root_mean_squared_error(true_values, predictions), true_values, predictions
        

In [12]:
rag = NoRagLLM(model_name="meta-llama/llama-3.2-1b-instruct:free")

2025-04-17 21:51:51,245 - INFO - Logger setup complete
2025-04-17 21:51:51,246 - INFO - NoRagLLM initialized with model: meta-llama/llama-3.2-1b-instruct:free


In [14]:
rmse, true_values, predictions = compute_metrics(df_test, rag)

  0%|          | 0/71 [00:00<?, ?it/s]2025-04-17 21:53:54,464 - INFO - Generating answer for query: Will the CDC's assessment of the risk posed by mpox to the US general public exceed "Very Low" before October 1, 2024?
2025-04-17 21:53:54,465 - INFO - Messages for LLM: [{'role': 'system', 'content': 'You are an expert forecaster. Given a binary event, estimate the probability (0-100%) that it will happen, and provide a brief justification for your estimate. Respond strictly in the following JSON format:\n{\n  "probability": <number>,\n  "justification": "<short explanation>"\n}\nDo not include any other text.'}, {'role': 'user', 'content': 'Event: Will the CDC\'s assessment of the risk posed by mpox to the US general public exceed "Very Low" before October 1, 2024?\nWhat is the probability this event will occur? Please follow the required JSON format.'}]
2025-04-17 21:53:55,729 - INFO - Response received: {'id': 'gen-1744916034-1nE5TeiVhauSZnf7wqAs', 'provider': 'Crusoe', 'model': 'met

Error: tuple indices must be integers or slices, not str. Retrying...


2025-04-17 21:53:57,019 - INFO - Response received: {'id': 'gen-1744916035-Z9WePy04adUc4YcDCN3b', 'provider': 'Crusoe', 'model': 'meta-llama/llama-3.2-1b-instruct', 'object': 'chat.completion', 'created': 1744916035, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "probability": "70",\n  "justification": "The CDC has already recently announced plans to proceed with assessing the risk of mpox in 2024, indicating an increasing level of activity toward the assessment process. This further escalates the likelihood that the risk assessment will exceed \'Very Low\' by the end of 2024."\n}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 151, 'completion_tokens': 72, 'total_tokens': 223, 'prompt_tokens_details': {'cached_tokens': 144}}}
2025-04-17 21:53:57,019 - INFO - Raw content: {
  "probability": "70",
  "justification": "The CDC has already recently announced plans to 

Error: tuple indices must be integers or slices, not str. Retrying...


2025-04-17 21:53:59,350 - INFO - Response received: {'id': 'gen-1744916037-DDBFK0brQJPfY87qcGyJ', 'provider': 'Crusoe', 'model': 'meta-llama/llama-3.2-1b-instruct', 'object': 'chat.completion', 'created': 1744916037, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "probability": 80,\n  "justification": "Based on the CDC\'s ongoing monitoring and evaluation of the mpox outbreak, including international surveillance results and the development of guidance for its prevention and control, it is reasonable to estimate a \'Low\' level of risk to the US general public by October 1, 2024."\n}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 150, 'completion_tokens': 72, 'total_tokens': 222, 'prompt_tokens_details': {'cached_tokens': 144}}}
2025-04-17 21:53:59,351 - INFO - Raw content: {
  "probability": 80,
  "justification": "Based on the CDC's ongoing monitoring and evalua

...

2025-04-17 21:59:07,606 - INFO - Response received: {'id': 'gen-1744916343-0vBnA6d2rs8JuUMQjYQA', 'provider': 'Crusoe', 'model': 'meta-llama/llama-3.2-1b-instruct', 'object': 'chat.completion', 'created': 1744916343, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "probability": 80,\n  "justification": "Production levels tend to follow a seasonal pattern, with Q3 being a peak quarter for automotive manufacturers. Historically, Tesla\'s production tends to increase in Q3, likely driven by holiday demand, inventory replenishment, and the need for new models to be completed. Additionally, Tesla\'s production cycle is generally more predictable, with a more consistent quarter-to-quarter production schedule compared to less predictable companies."\n}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 139, 'completion_tokens': 95, 'total_tokens': 234, 'prompt_tokens_details'

Error: tuple indices must be integers or slices, not str. Retrying...


2025-04-17 21:59:15,439 - INFO - Response received: {'id': 'gen-1744916353-9agj8hl0WCUUtXqIK7Cw', 'provider': 'Crusoe', 'model': 'meta-llama/llama-3.2-1b-instruct', 'object': 'chat.completion', 'created': 1744916353, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "probability": 50,\n  "justification": "A tight general election is expected, but there is no predictive indicator from these polls that suggests that Keir Starmer\'s approval rating will be less than 40 on September 18, 2024."', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 149, 'completion_tokens': 56, 'total_tokens': 205, 'prompt_tokens_details': {'cached_tokens': 96}}}
2025-04-17 21:59:15,440 - INFO - Raw content: {
  "probability": 50,
  "justification": "A tight general election is expected, but there is no predictive indicator from these polls that suggests that Keir Starmer's approval rating will b

Error: tuple indices must be integers or slices, not str. Retrying...


2025-04-17 21:59:18,313 - INFO - Response received: {'id': 'gen-1744916355-xKZ8KOn4MfvA2tVv6N8t', 'provider': 'Crusoe', 'model': 'meta-llama/llama-3.2-1b-instruct', 'object': 'chat.completion', 'created': 1744916355, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "probability": 0,\n  "justification": ""\n}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 149, 'completion_tokens': 16, 'total_tokens': 165, 'prompt_tokens_details': {'cached_tokens': 96}}}
2025-04-17 21:59:18,314 - INFO - Raw content: {
  "probability": 0,
  "justification": ""
}
2025-04-17 21:59:18,315 - INFO - Parsed response: {'probability': 0, 'justification': ''}
 97%|█████████▋| 69/71 [05:23<00:11,  5.80s/it]2025-04-17 21:59:18,317 - INFO - Generating answer for query: Will Keir Starmer's approval rating as Prime Minister of the United Kingdom be less than 60 on September 18, 2024?
2025-04-17 21:5

In [15]:
print(f"RMSE: {rmse}")
print(f"True values: {true_values}")
print(f"Predictions: {predictions}")

RMSE: 38.27116187138712
True values: [14, 15, 38, 99, 54, 20, 65, 60, 20, 5, 85, 82, 0, 15, 18, 18, 1, 18, 1, 33, 30, 16, 49, 24, 25, 73, 63, 65, 30, 5, 17, 25, 5, 50, 44, 75, 10, 40, 60, 2, 48, 45, 40, 63, 45, 57, 15, 37, 22, 85, 38, 65, 65, 50, 41, 13, 15, 80, 77, 0, 20, 45, 59, 75, 70, 1, 60, 1, 65, 83, 30]
Predictions: ['70', 80, 0, 25, 5, 30, 30, 30, 30, 70.4, 20, 0, 30, 0, 30, 0, 50, 11, 93, 20, '20', 20, 25, 20, 25, 0, 30, 5, 0, 0, 40, 78, 20, 80, 20, 20, 5, '50', 65, 15, 1.5, 80, 10, 60, 20, 20, 10, 70, 20, 20, 61, 45, 44, 15, 20, 80, 60, 80, 100, 0, 30, 85, 75, 20, 0, 20, 80, 20, 0, 80, 20]
